In [1]:
import os
import pickle
import json
import csv
import time
import lyricsgenius as lg
import re
import numpy as np
import pandas as pd

#import credential
import requests

In [2]:
CLIENT_ACCESS_TOKEN = "xXeju4S5sFpDZOv5SAjWMwmOjcjTChruhQj6o3rK6O5ROA5mlA9MLx0Zth4S8ljG"
genius = lg.Genius(CLIENT_ACCESS_TOKEN)


Degrees of similarity for lyrics data:
- annotation text and the respective lyrics text portion should have high similarity

In [3]:
def lyrics_annotations_pairs_gen(genius_song_id):

    annotations_sample_song = genius.song_annotations(genius_song_id)

    l_pairs = []

    for annotation_pair in annotations_sample_song:

        lyrics_ref = annotation_pair[0]

        cond_empty_ref = len(lyrics_ref) <1 or lyrics_ref.isspace()
        cond_ref_chorus = re.match(r"(?i)^\[chorus\s*\d*\]$", lyrics_ref)
        cond_ref_verse = re.match(r"(?i)^\[verse\s*\d*\]$", lyrics_ref)
        
        # Ignore hanlding for "empty reference"
        if cond_empty_ref or cond_ref_chorus or cond_ref_verse:

            continue

        annotation = annotation_pair[1][0][0]
        t_pair = (lyrics_ref, annotation)
        l_pairs.append(t_pair)

    return l_pairs

In [6]:
with open('../App/pickle_objects/annot_df.pickle', 'rb') as f:
        df_train = pickle.load(f)
s_songs_id = pd.to_numeric(df_train['genius_id'], errors='coerce')
list_songs_id = s_songs_id.astype('Int64').to_list()
df_with_annotation = df_train.astype('str')[df_train['annotation']!="[]"]
len(df_with_annotation)

1000

In [8]:
from finetune_pairs import lyrics_annotations_gen_id
l_pairs, l_genius_song_idx = lyrics_annotations_gen_id(list_songs_id)

In [9]:
len(l_pairs)

1137

In [10]:
len(l_genius_song_idx)

1137

In [15]:
# inspect the first 5 annotation-lyrics pairs
l_pairs[:5]

[('I ain’t afraid to act like a fool\n Like a motherfuckin kid',
  'This is a continuation of a motif SkyBlu has referenced throughout his entire musical career. The motif is of SkyBlu being a child or being immature either in his simplistic lyricism or subject matter.'),
 ('We gonna do what they say can’t be done',
  'The meaning of this line is many men have tried moving Coors but have failed!'),
 ('Loaded up and truckin’',
  'In the movie they got the beer and were headed back to Atlanta GA.'),
 ('Let it all hang out ’cause we’ve got a run to make',
  '“Let it all hang out” is slang for “don’t hold back and be yourself”, and its literal meaning comes from the notion of being naked and allowing the male genitalia to “hang” freely, although when used in modern English, this is not usually a literal request.\n\nThe line restates the urgency of the delivery and the need to focus on the task at hand.'),
 ('You’ve gotta keep that diesel truckin’',
  'The vast majority of heavy goods vehic

In [16]:
l_pickle = [l_pairs, l_genius_song_idx]
with open('../App/pickle_objects/lyrics_annotation_pairs_train.pickle', 'wb') as f:
    pickle.dump(l_pickle, f)

Other dimensions to be considered

- lyrics text with same artist/ same song-writer to have high similarity
- lyrics text in the same song to have high similarity
- lyrics text in the same verse should have high similarity